In [1]:
import icalendar
import pytz, zoneinfo, dateutil.tz  # timezone libraries
import datetime, icalendar
import pyaml
import pandas as pd 
import numpy as np
import yaml
from datetime import datetime, timedelta
import re
from ics import Calendar, Event

In [2]:



def convert_datestring (datestring):
    format = "%m/%d/%Y"
    date_object = datetime.strptime(datestring, format)
    return date_object


def get_start_end_dates(block_id, my_dict):
    start, end = (my_dict['block_dates'][block_id].split())
    start = convert_datestring(start)
    end = convert_datestring(end)
    return start, end


def date_lists(start_date, end_date):
    current_date = start_date
    dates = []
    while current_date <= end_date:
        dates.append(current_date)
        current_date += timedelta(days=1)
    return dates
    
    
def get_specific_rotation_schedule(block_id, rotation, my_dict):
    rotation = re.sub('\s', '', rotation)
    

        
    try:
        rotation, individual = rotation.split('-')
        individual = individual[0] # gets the first character of an individual (A, B, C, D, E, F etc.)
    except:
        rotation = rotation
        individual = 'Z'
        
    if re.search('Janeway|Barker|Longcope|Thayer', rotation):
        rotation_category = 'O'    
    elif re.search('CJ|Polk|MEG', rotation):
        rotation_category = 'MEG_CJ_POLK'    
        
    elif re.search('PCCU|Addiction', rotation):
        # these rotation have MTWTF rotation with weekends off
        rotation_category = 'Ambulatory'
    else:
        rotation_category = rotation
        
    for schedule_type, schedule in my_dict[rotation_category].items():
        if block_id in schedule['blocks']:
            return schedule[individual].split()
        
        
        
def block_dates (block_id, my_dict):
    """get start and end date for each block by going through the master yaml """
    start_date, end_date = (get_start_end_dates(block_id, my_dict))
    block_duration = date_lists(start_date, end_date)    
    return block_duration
 

def add_date_postcall_into_next_block(call_cycle, block_days):
    print(call_cycle[-1])
    if re.search(r'^\*', call_cycle[-1]):
        # Step 1: Get the last date in the list
        last_date = block_days[-1]

        # Step 2: Calculate the next date (assuming consecutive days, so add 1 day)
        next_date = last_date + timedelta(days=1)

        # Step 3: Add the new date to the list
        block_days.append(next_date)
    return block_days
    



def oncology_schedule(rotation, date):
    schedule = 'DAY'
    if date.weekday() == 5 and rotation in ['Solids', 'Leuks']: #off day is Sunday -> changed to Saturday in 2025
        schedule = 'OFF'
    elif date.weekday() == 6 and rotation == 'MTL': # off day is Saturday -> changed to Sunday in 2025
        schedule = 'OFF'
    elif date.weekday() == 5 and rotation in ['Ambulatory', 'UCM', 'AddictionMedicine', 'Psych']: #ambo off day is Saturday and Sunday
        schedule = 'OFF'   
    elif date.weekday() == 6 and rotation in ['Ambulatory', 'UCM', 'AddictionMedicine', 'Psych']: #ambo off day is Saturday and Sunday
        schedule = 'OFF'
    elif date.weekday() == 6 and rotation == 'Subspecialty': # subspecialty intern off on Sunday
        schedule = 'OFF'
    else:
        pass
    
    return schedule    

In [3]:
# Read master tables

    
with open("/Users/cjyoon/Dropbox/Osler/Osler_shiny/intern2025/call_schedule_master.yaml") as f:
    my_dict = yaml.safe_load(f)

    
master_schedule = pd.read_excel('/Users/cjyoon/Dropbox/Osler/Osler_shiny/intern2025/Interns 2025-2026.xlsx', sheet_name='Intern Schedule')
master_schedule['First Name'] = master_schedule['First Name'].fillna('')
master_schedule['name'] = master_schedule['Last Name'].astype(str) + '_' + master_schedule['First Name'].astype(str)

# Drop the original 'Last Name' and 'First Name' columns
master_schedule = master_schedule.drop(columns=['Last Name', 'First Name'])

master_schedule = master_schedule[['name'] + [col for col in master_schedule.columns if col != 'name']]
print(master_schedule)


# Remove NaN values
cleaned_series = master_schedule['name'].dropna()

# Convert back to a list
resident_names = cleaned_series.tolist()
resident_names = [name for name in resident_names if not str(name).startswith('nan')]
resident_names = [re.sub(r'_nan$', '', str(name)) for name in resident_names]


print(resident_names)

                         name                   1A                   1B  \
0              nan_Date Start  2025-07-01 00:00:00  2025-07-13 00:00:00   
1                nan_Date End  2025-07-12 00:00:00  2025-07-23 00:00:00   
2   Abdul Rahman Lebbe_Ahamed           Ambulatory                   CJ   
3               Agrawal_Rashi         Longcope - C         Longcope - C   
4       Amoo-Mitchual_Sheldon              MPC - A           Ambulatory   
..                        ...                  ...                  ...   
57                Watts_Haley         Longcope - A         Longcope - A   
58               Wolfe_Isabel           Barker - B           Barker - B   
59                     Yu_Jia             MICU - A         Brancati - C   
60                 Zhang_Bill              CCU - C                Leuks   
61                  Zhao_Luke         Longcope - B         Longcope - B   

                     2A                   2B                   3A  \
0   2025-07-24 00:00:00  2025-

In [4]:
def get_schedule(resident_name, master_schedule):
    """gets the specific intern's schedule from the table"""
    filtered_df = master_schedule[master_schedule.iloc[:, 0] == resident_name]
    return (filtered_df.squeeze().to_list()[1:])



In [5]:
master_schedule

# Now construct new column names
block_labels = master_schedule.columns[0:]  # Exclude 'name'
start_dates = master_schedule.iloc[0, 0:]
end_dates = master_schedule.iloc[1, 0:]

# Start with the first column unchanged
new_columns = []

# Process each remaining column individually
for col, start, end in zip(block_labels, start_dates, end_dates):
    try:
        start_str = pd.to_datetime(start).strftime('%-m/%-d/%Y')
        end_str = pd.to_datetime(end).strftime('%-m/%-d/%Y')
        new_columns.append(f"{col} {start_str}-{end_str}")
    except Exception:
        # Fallback to original column name if any error occurs
        new_columns.append(col)


master_schedule.columns =  new_columns

# Drop the first two rows (date start and end)
master_schedule = master_schedule.iloc[2:].reset_index(drop=True)
print(master_schedule)

                         name 1A 7/1/2025-7/12/2025 1B 7/13/2025-7/23/2025  \
0   Abdul Rahman Lebbe_Ahamed            Ambulatory                     CJ   
1               Agrawal_Rashi          Longcope - C           Longcope - C   
2       Amoo-Mitchual_Sheldon               MPC - A             Ambulatory   
3               Anesthesia_1_          Brancati - B           Brancati - B   
4        Baffoe-Bonnie_Helena                   MEG             Ambulatory   
5             Baggett_Katelyn          Brancati - D             Ambulatory   
6                Chalom_Mayer           Janeway - D            Janeway - D   
7              Chaudhry_Rahul            Ambulatory               MICU - B   
8                Chishti_Emad            Ambulatory                MPC - C   
9                    Dang_Nhu            Barker - D             Barker - D   
10              DeLuca_Marisa           Janeway - C            Janeway - C   
11           Dubhashi_Janhavi                  Peds             

In [6]:
master_table_tsv = master_schedule.iloc[3:].fillna('')
master_table_tsv.replace("Subspecialty", "NightWatch", inplace=True)

master_table_tsv.sort_values('name').to_csv('/Users/cjyoon/Dropbox/Osler/Osler_shiny/block_schedule/intern_block_view2025.tsv', sep='\t', index=False)

In [7]:
master_table_tsv.sort_values('name')


,name,1A 7/1/2025-7/12/2025,1B 7/13/2025-7/23/2025,2A 7/24/2025-8/6/2025,2B 8/7/2025-8/20/2025,3A 8/21/2025-9/3/2025,3B 9/4/2025-9/17/2025,4A 9/18/2025-10/1/2025,4B 10/2/2025-10/15/2025,5A 10/16/2025-10/29/2025,...,9A 2/12/2026-2/25/2026,9B 2/26/2026-3/11/2026,10A 3/12/2026-3/25/2026,10B 3/26/2026-4/8/2026,11A 4/9/2026-4/22/2026,11B 4/23/2026-5/6/2026,12A 5/7/2026-5/20/2026,12B 5/21/2026-6/3/2026,13A 6/4/2026-6/17/2026,13B 6/18/2026-6/30/2026
3,Anesthesia_1_,Brancati - B,Brancati - B,Brancati - B,Brancati - B,Brancati - B,Brancati - B,Brancati - B,Brancati - B,Brancati - B,...,Brancati,Brancati,Brancati,Brancati,Brancati,Brancati,Brancati,Brancati,Brancati,Brancati
4,Baffoe-Bonnie_Helena,MEG,Ambulatory,Longcope - B,Longcope - B,Vacation,Ambulatory,MPC - A,Solids,MICU - A,...,Longcope,Ambulatory,Vacation,Longcope,Longcope,Ambulatory,MICU,ED,MPC,Ambulatory
5,Baggett_Katelyn,Brancati - D,Ambulatory,Janeway - C,Janeway - C,MPC - B,Ambulatory,MICU - B,Janeway - D,CCU - C,...,Janeway,Ambulatory,MPC,Vacation,MICU,Ambulatory,Solids,Janeway,Janeway,Ambulatory
6,Chalom_Mayer,Janeway - D,Janeway - D,Ambulatory,MICU - F,Brancati - A,Vacation,Ambulatory,Janeway - C,Janeway - C,...,NightWatch,MICU,Ambulatory,Janeway,MTL,MPC,Ambulatory,Vacation,Janeway,Janeway
7,Chaudhry_Rahul,Ambulatory,MICU - B,ED,CCU - A,Ambulatory,Brancati - A,Barker - C,Barker - C,Ambulatory,...,Ambulatory,Barker,Barker,MPC,Ambulatory,ED,Vacation,Barker,Ambulatory,Barker
8,Chishti_Emad,Ambulatory,MPC - C,Janeway - D,Janeway - D,Ambulatory,Janeway - A,ED,Vacation,Ambulatory,...,Ambulatory,MTL,Brancati,MICU,Ambulatory,Janeway,Janeway,MPC,Ambulatory,Janeway
9,Dang_Nhu,Barker - D,Barker - D,MPC - C,Ambulatory,Vacation,MICU - C,Solids,Ambulatory,Barker - A,...,MICU,Leuks,MPC,Ambulatory,ED,Barker,Barker,Ambulatory,MICU,Vacation
10,DeLuca_Marisa,Janeway - C,Janeway - C,Ambulatory,Solids,MICU - B,ED,Ambulatory,MPC - B,Vacation,...,Janeway,Janeway,Ambulatory,Brancati,MICU,Vacation,Ambulatory,MTL,Janeway,Janeway
11,Dubhashi_Janhavi,Peds,Peds,Peds,MP Clinic,Barker - B,Barker - B,CCU - A,MP Clinic,Peds,...,Barker,Barker,Peds,Peds,Peds,Peds,Brancati,MP Clinic,Barker,Barker
12,ED_1_,,,CCU - B,CCU - B,CCU - B,CCU - B,CCU - B,CCU - B,CCU - B,...,,,,,,,,,,


In [8]:
for resident_name in resident_names:
    date_list = []
    schedule_list = []        
    my_blocks = (get_schedule(resident_name, master_schedule))
    print(my_blocks)
    

    calendar = Calendar()
    event = Event()
    with open(f"/Users/cjyoon/Dropbox/Osler/Osler_shiny/intern2025/schedule/{resident_name}_schedule.tsv", "w") as g:
        g.write('name\tdate\tschedule\n')
        for block_id, rotation in zip(my_dict['block_dates'].keys(), my_blocks):
            # only write out upto holiday block 2 since remaining scheudle has not been finalized yet
#             if block_id in ['7B', '8A', '8B', '9A', '9B', '10A', '10B', '11A', '11B', '12A', '12B', '13A', '13B']:
#             if block_id in ['1A', '1B', '2A', '2B', '3A', '3B', '4A', '4B', '5A', '5B', '6A', '6B', '7A', 'Holiday 1', 'Holiday2']:
            if 1:
        
                if not isinstance(rotation, str):
                    rotation = ' '
                
                rotation = re.sub('\s', '', rotation)

                print(block_id, rotation)

                block_days = block_dates(block_id, my_dict)        

                try:
                    call_cycle = get_specific_rotation_schedule(block_id, rotation, my_dict)
                except:
                    # if specific call cycle is unknown for a rotation, then default to unknown so manual creation can be made
                                        # if specific call cycle is unknown for a rotation, then default to unknown so manual creation can be made
                    if rotation == ' ':
                        call_cycle = [' '] * 14
                    else:
                        call_cycle = ['-'] * 14

                if call_cycle == None:
                    call_cycle = ['-'] * 14


                print(f'{block_days} {len(block_days)}')
                print(f'{call_cycle} {len(call_cycle)}')


                #if block_id == '1A':
                #    # create call cycle that is offset to account for different start date for july1 cycle
                #    del call_cycle[:july1_offset]

                # check if block call cycle ends in a post call
                block_days = add_date_postcall_into_next_block(call_cycle, block_days)

                # Loop through each date and corresponding schedule
                for date, schedule in zip(block_days, call_cycle):
                    
                    if rotation in ['MTL', 'Leuks', 'Solids', 'Ambulatory','Psych', 'UCM', 'AddictionMedicine']:
                        schedule = oncology_schedule(rotation, date)
                    
                    event = Event()  # Create a new event object inside the loop
                    if rotation == 'Subspecialty': # change Subspecialty into NightWatch more commonly referred.
                        rotation = 'NightWatch'
                    
                    if rotation == 'MPClinic':
                        rotation = 'MP_Clinic'

                    event.name = f'{block_id} {rotation} {schedule}'
                    event.begin = date
                    event.make_all_day()  # Make the event an all-day event
                    print(date, block_id, rotation, schedule)
                    # Add event to calendar
                    calendar.events.add(event)
                    date_string = date.strftime('%m-%d-%Y')
                    g.write(f'{resident_name}\t{date_string}\t{event.name}\n')



        # Write the calendar to a .ics file
        with open(f"/Users/cjyoon/Dropbox/Osler/Osler_shiny/www/schedule2025//{resident_name}_schedule.ics", "w") as f:
            f.writelines(calendar.serialize_iter())

    

['Ambulatory', 'CJ', 'MTL', 'MICU - B', 'Ambulatory', 'Janeway - C', 'Janeway - C', 'Vacation', 'Ambulatory', 'Janeway - C', 'Janeway - C', 'ED', 'Ambulatory', 'Janeway - A', 'Off', 'Janeway', 'Janeway', 'CCU', 'Ambulatory', 'MPC', 'Vacation', 'MICU', 'Ambulatory', 'Solids', 'Janeway', 'Janeway', 'Ambulatory', 'MICU']
1A Ambulatory
[datetime.datetime(2025, 7, 1, 0, 0), datetime.datetime(2025, 7, 2, 0, 0), datetime.datetime(2025, 7, 3, 0, 0), datetime.datetime(2025, 7, 4, 0, 0), datetime.datetime(2025, 7, 5, 0, 0), datetime.datetime(2025, 7, 6, 0, 0), datetime.datetime(2025, 7, 7, 0, 0), datetime.datetime(2025, 7, 8, 0, 0), datetime.datetime(2025, 7, 9, 0, 0), datetime.datetime(2025, 7, 10, 0, 0), datetime.datetime(2025, 7, 11, 0, 0), datetime.datetime(2025, 7, 12, 0, 0)] 12
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'] 14
-
2025-07-01 00:00:00 1A Ambulatory DAY
2025-07-02 00:00:00 1A Ambulatory DAY
2025-07-03 00:00:00 1A Ambulatory DAY
2025-07-04 00:00:00 1A Am

2025-09-16 00:00:00 3B Ambulatory DAY
2025-09-17 00:00:00 3B Ambulatory DAY
4A MICU-B
[datetime.datetime(2025, 9, 18, 0, 0), datetime.datetime(2025, 9, 19, 0, 0), datetime.datetime(2025, 9, 20, 0, 0), datetime.datetime(2025, 9, 21, 0, 0), datetime.datetime(2025, 9, 22, 0, 0), datetime.datetime(2025, 9, 23, 0, 0), datetime.datetime(2025, 9, 24, 0, 0), datetime.datetime(2025, 9, 25, 0, 0), datetime.datetime(2025, 9, 26, 0, 0), datetime.datetime(2025, 9, 27, 0, 0), datetime.datetime(2025, 9, 28, 0, 0), datetime.datetime(2025, 9, 29, 0, 0), datetime.datetime(2025, 9, 30, 0, 0), datetime.datetime(2025, 10, 1, 0, 0)] 14
['POST', 'GOOD', 'CALL', 'POST', 'OFF', 'CALL', 'POST', 'GOOD', 'CALL', 'POST', 'OFF', 'CALL', 'POST', 'GOOD'] 14
GOOD
2025-09-18 00:00:00 4A MICU-B POST
2025-09-19 00:00:00 4A MICU-B GOOD
2025-09-20 00:00:00 4A MICU-B CALL
2025-09-21 00:00:00 4A MICU-B POST
2025-09-22 00:00:00 4A MICU-B OFF
2025-09-23 00:00:00 4A MICU-B CALL
2025-09-24 00:00:00 4A MICU-B POST
2025-09-25 00:0

['Janeway - D', 'Janeway - D', 'Ambulatory', 'MICU - F', 'Brancati - A', 'Vacation', 'Ambulatory', 'Janeway - C', 'Janeway - C', 'MPC - A', 'Ambulatory', 'ED', 'MICU - C', 'MICU - C', 'Off', 'CCU', 'Ambulatory', 'Leuks', 'Subspecialty', 'MICU', 'Ambulatory', 'Janeway', 'MTL', 'MPC', 'Ambulatory', 'Vacation', 'Janeway', 'Janeway']
1A Janeway-D
[datetime.datetime(2025, 7, 1, 0, 0), datetime.datetime(2025, 7, 2, 0, 0), datetime.datetime(2025, 7, 3, 0, 0), datetime.datetime(2025, 7, 4, 0, 0), datetime.datetime(2025, 7, 5, 0, 0), datetime.datetime(2025, 7, 6, 0, 0), datetime.datetime(2025, 7, 7, 0, 0), datetime.datetime(2025, 7, 8, 0, 0), datetime.datetime(2025, 7, 9, 0, 0), datetime.datetime(2025, 7, 10, 0, 0), datetime.datetime(2025, 7, 11, 0, 0), datetime.datetime(2025, 7, 12, 0, 0)] 12
['GOOD', 'CALL', 'POST', 'DAY', 'OFF', 'CALL', 'POST', 'DAY', 'GOOD', 'CALL', 'POST', 'DAY'] 12
DAY
2025-07-01 00:00:00 1A Janeway-D GOOD
2025-07-02 00:00:00 1A Janeway-D CALL
2025-07-03 00:00:00 1A Janew

2026-01-19 00:00:00 8A Ambulatory DAY
2026-01-20 00:00:00 8A Ambulatory DAY
2026-01-21 00:00:00 8A Ambulatory DAY
2026-01-22 00:00:00 8A Ambulatory DAY
2026-01-23 00:00:00 8A Ambulatory DAY
2026-01-24 00:00:00 8A Ambulatory OFF
2026-01-25 00:00:00 8A Ambulatory OFF
2026-01-26 00:00:00 8A Ambulatory DAY
2026-01-27 00:00:00 8A Ambulatory DAY
2026-01-28 00:00:00 8A Ambulatory DAY
8B Leuks
[datetime.datetime(2026, 1, 29, 0, 0), datetime.datetime(2026, 1, 30, 0, 0), datetime.datetime(2026, 1, 31, 0, 0), datetime.datetime(2026, 2, 1, 0, 0), datetime.datetime(2026, 2, 2, 0, 0), datetime.datetime(2026, 2, 3, 0, 0), datetime.datetime(2026, 2, 4, 0, 0), datetime.datetime(2026, 2, 5, 0, 0), datetime.datetime(2026, 2, 6, 0, 0), datetime.datetime(2026, 2, 7, 0, 0), datetime.datetime(2026, 2, 8, 0, 0), datetime.datetime(2026, 2, 9, 0, 0), datetime.datetime(2026, 2, 10, 0, 0), datetime.datetime(2026, 2, 11, 0, 0)] 14
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'] 14
-
2026-01-

2026-03-11 00:00:00 9B Barker -
10A Peds
[datetime.datetime(2026, 3, 12, 0, 0), datetime.datetime(2026, 3, 13, 0, 0), datetime.datetime(2026, 3, 14, 0, 0), datetime.datetime(2026, 3, 15, 0, 0), datetime.datetime(2026, 3, 16, 0, 0), datetime.datetime(2026, 3, 17, 0, 0), datetime.datetime(2026, 3, 18, 0, 0), datetime.datetime(2026, 3, 19, 0, 0), datetime.datetime(2026, 3, 20, 0, 0), datetime.datetime(2026, 3, 21, 0, 0), datetime.datetime(2026, 3, 22, 0, 0), datetime.datetime(2026, 3, 23, 0, 0), datetime.datetime(2026, 3, 24, 0, 0), datetime.datetime(2026, 3, 25, 0, 0)] 14
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'] 14
-
2026-03-12 00:00:00 10A Peds -
2026-03-13 00:00:00 10A Peds -
2026-03-14 00:00:00 10A Peds -
2026-03-15 00:00:00 10A Peds -
2026-03-16 00:00:00 10A Peds -
2026-03-17 00:00:00 10A Peds -
2026-03-18 00:00:00 10A Peds -
2026-03-19 00:00:00 10A Peds -
2026-03-20 00:00:00 10A Peds -
2026-03-21 00:00:00 10A Peds -
2026-03-22 00:00:00 10A Peds -
2026-0

...

['Janeway - B', 'Janeway - B', 'MPC - B', 'Ambulatory', 'Janeway - B', 'Janeway - B', 'Vacation', 'Ambulatory', 'MICU - B', 'MTL', 'ED', 'Ambulatory', 'Brancati - A', 'Off', 'Janeway - A', 'Janeway', 'MICU', 'Ambulatory', 'ED', 'Vacation', 'CCU', 'Ambulatory', 'Janeway', 'Janeway', 'MPC', 'Ambulatory', 'MICU', 'Solids']
1A Janeway-B
[datetime.datetime(2025, 7, 1, 0, 0), datetime.datetime(2025, 7, 2, 0, 0), datetime.datetime(2025, 7, 3, 0, 0), datetime.datetime(2025, 7, 4, 0, 0), datetime.datetime(2025, 7, 5, 0, 0), datetime.datetime(2025, 7, 6, 0, 0), datetime.datetime(2025, 7, 7, 0, 0), datetime.datetime(2025, 7, 8, 0, 0), datetime.datetime(2025, 7, 9, 0, 0), datetime.datetime(2025, 7, 10, 0, 0), datetime.datetime(2025, 7, 11, 0, 0), datetime.datetime(2025, 7, 12, 0, 0)] 12
['POST', 'DAY', 'GOOD', 'CALL', 'POST', 'DAY', 'OFF', 'CALL', 'POST', 'DAY', 'OFF', 'CALL', '*POST'] 13
*POST
2025-07-01 00:00:00 1A Janeway-B POST
2025-07-02 00:00:00 1A Janeway-B DAY
2025-07-03 00:00:00 1A Janewa

2026-06-14 00:00:00 13A MICU -
2026-06-15 00:00:00 13A MICU -
2026-06-16 00:00:00 13A MICU -
2026-06-17 00:00:00 13A MICU -
13B Solids
[datetime.datetime(2026, 6, 18, 0, 0), datetime.datetime(2026, 6, 19, 0, 0), datetime.datetime(2026, 6, 20, 0, 0), datetime.datetime(2026, 6, 21, 0, 0), datetime.datetime(2026, 6, 22, 0, 0), datetime.datetime(2026, 6, 23, 0, 0), datetime.datetime(2026, 6, 24, 0, 0), datetime.datetime(2026, 6, 25, 0, 0), datetime.datetime(2026, 6, 26, 0, 0), datetime.datetime(2026, 6, 27, 0, 0), datetime.datetime(2026, 6, 28, 0, 0), datetime.datetime(2026, 6, 29, 0, 0), datetime.datetime(2026, 6, 30, 0, 0)] 13
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'] 14
-
2026-06-18 00:00:00 13B Solids DAY
2026-06-19 00:00:00 13B Solids DAY
2026-06-20 00:00:00 13B Solids OFF
2026-06-21 00:00:00 13B Solids DAY
2026-06-22 00:00:00 13B Solids DAY
2026-06-23 00:00:00 13B Solids DAY
2026-06-24 00:00:00 13B Solids DAY
2026-06-25 00:00:00 13B Solids DAY
2026-06-26 

2025-11-14 00:00:00 6A Ambulatory DAY
2025-11-15 00:00:00 6A Ambulatory OFF
2025-11-16 00:00:00 6A Ambulatory OFF
2025-11-17 00:00:00 6A Ambulatory DAY
2025-11-18 00:00:00 6A Ambulatory DAY
2025-11-19 00:00:00 6A Ambulatory DAY
2025-11-20 00:00:00 6A Ambulatory DAY
2025-11-21 00:00:00 6A Ambulatory DAY
2025-11-22 00:00:00 6A Ambulatory OFF
2025-11-23 00:00:00 6A Ambulatory OFF
2025-11-24 00:00:00 6A Ambulatory DAY
2025-11-25 00:00:00 6A Ambulatory DAY
2025-11-26 00:00:00 6A Ambulatory DAY
6B MICU-C
[datetime.datetime(2025, 11, 27, 0, 0), datetime.datetime(2025, 11, 28, 0, 0), datetime.datetime(2025, 11, 29, 0, 0), datetime.datetime(2025, 11, 30, 0, 0), datetime.datetime(2025, 12, 1, 0, 0), datetime.datetime(2025, 12, 2, 0, 0), datetime.datetime(2025, 12, 3, 0, 0), datetime.datetime(2025, 12, 4, 0, 0), datetime.datetime(2025, 12, 5, 0, 0), datetime.datetime(2025, 12, 6, 0, 0), datetime.datetime(2025, 12, 7, 0, 0), datetime.datetime(2025, 12, 8, 0, 0), datetime.datetime(2025, 12, 9, 0, 0

['Longcope - B', 'Longcope - B', 'ED', 'Ambulatory', 'Longcope - B', 'Longcope - B', 'MTL', 'Ambulatory', 'MICU - C', 'Brancati - D', 'Vacation', 'Ambulatory', 'MICU - A', 'MICU - A', 'Off', 'CCU', 'Longcope ', 'Ambulatory', 'Solids', 'Vacation', 'MICU', 'Ambulatory', 'Longcope', 'Longcope', 'MPC', 'Ambulatory', 'ED', 'Longcope']
1A Longcope-B
[datetime.datetime(2025, 7, 1, 0, 0), datetime.datetime(2025, 7, 2, 0, 0), datetime.datetime(2025, 7, 3, 0, 0), datetime.datetime(2025, 7, 4, 0, 0), datetime.datetime(2025, 7, 5, 0, 0), datetime.datetime(2025, 7, 6, 0, 0), datetime.datetime(2025, 7, 7, 0, 0), datetime.datetime(2025, 7, 8, 0, 0), datetime.datetime(2025, 7, 9, 0, 0), datetime.datetime(2025, 7, 10, 0, 0), datetime.datetime(2025, 7, 11, 0, 0), datetime.datetime(2025, 7, 12, 0, 0)] 12
['POST', 'DAY', 'GOOD', 'CALL', 'POST', 'DAY', 'OFF', 'CALL', 'POST', 'DAY', 'OFF', 'CALL', '*POST'] 13
*POST
2025-07-01 00:00:00 1A Longcope-B POST
2025-07-02 00:00:00 1A Longcope-B DAY
2025-07-03 00:00

2026-02-03 00:00:00 8B Ambulatory DAY
2026-02-04 00:00:00 8B Ambulatory DAY
2026-02-05 00:00:00 8B Ambulatory DAY
2026-02-06 00:00:00 8B Ambulatory DAY
2026-02-07 00:00:00 8B Ambulatory OFF
2026-02-08 00:00:00 8B Ambulatory OFF
2026-02-09 00:00:00 8B Ambulatory DAY
2026-02-10 00:00:00 8B Ambulatory DAY
2026-02-11 00:00:00 8B Ambulatory DAY
9A Solids
[datetime.datetime(2026, 2, 12, 0, 0), datetime.datetime(2026, 2, 13, 0, 0), datetime.datetime(2026, 2, 14, 0, 0), datetime.datetime(2026, 2, 15, 0, 0), datetime.datetime(2026, 2, 16, 0, 0), datetime.datetime(2026, 2, 17, 0, 0), datetime.datetime(2026, 2, 18, 0, 0), datetime.datetime(2026, 2, 19, 0, 0), datetime.datetime(2026, 2, 20, 0, 0), datetime.datetime(2026, 2, 21, 0, 0), datetime.datetime(2026, 2, 22, 0, 0), datetime.datetime(2026, 2, 23, 0, 0), datetime.datetime(2026, 2, 24, 0, 0), datetime.datetime(2026, 2, 25, 0, 0)] 14
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'] 14
-
2026-02-12 00:00:00 9A Solids DAY
20

Zhao_Luke 13B Longcope ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']


In [10]:
                
                rotation = re.sub('\s', '', rotation)

                print(block_id, rotation)

                block_days = block_dates(block_id, my_dict)        
                print(block_days)
                try:
                    call_cycle = get_specific_rotation_schedule(block_id, rotation, my_dict)
                    print(call_cycle)
                except:
                    # if specific call cycle is unknown for a rotation, then default to unknown so manual creation can be made
                                        # if specific call cycle is unknown for a rotation, then default to unknown so manual creation can be made
                    if rotation == ' ':
                        call_cycle = [' '] * 14
                    else:
                        call_cycle = ['-'] * 14
                        print(call_cycle)

13B Longcope
[datetime.datetime(2026, 6, 18, 0, 0), datetime.datetime(2026, 6, 19, 0, 0), datetime.datetime(2026, 6, 20, 0, 0), datetime.datetime(2026, 6, 21, 0, 0), datetime.datetime(2026, 6, 22, 0, 0), datetime.datetime(2026, 6, 23, 0, 0), datetime.datetime(2026, 6, 24, 0, 0), datetime.datetime(2026, 6, 25, 0, 0), datetime.datetime(2026, 6, 26, 0, 0), datetime.datetime(2026, 6, 27, 0, 0), datetime.datetime(2026, 6, 28, 0, 0), datetime.datetime(2026, 6, 29, 0, 0), datetime.datetime(2026, 6, 30, 0, 0)]
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']


In [11]:
def authenticate_google_drive():
    creds = None

    # Use token.json to store the user's access and refresh tokens
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    
    # If there are no valid credentials, let the user log in and authorize the app
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # Use the client_secret.json to start the OAuth flow
            flow = InstalledAppFlow.from_client_secrets_file(
                '/Users/cjyoon/Dropbox/Osler/osler/client_secret_257342927366-4q3itidpqit3vcehjca5rvlg9m2evhsi.apps.googleusercontent.com.json', SCOPES)
            creds = flow.run_local_server(port=0)
        
        # Save the credentials for future runs in token.json
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    # Build the Google Drive service object
    service = build('drive', 'v3', credentials=creds)
    return service



def upload_ics_to_drive(file_path):
    service = authenticate_google_drive()

    # Upload the file to Google Drive with the correct MIME type for .ics files
    file_metadata = {'name': os.path.basename(file_path), 'mimeType': 'text/calendar'}
    media = MediaFileUpload(file_path, mimetype='text/calendar', parent='osler_schedule' )
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()

    # Make the file public and get the shareable link
    file_id = file.get('id')
    service.permissions().create(fileId=file_id, body={'type': 'anyone', 'role': 'reader'}).execute()
    shareable_url = f"https://drive.google.com/file/d/osler_schedule/{file_id}/view"
    
    return shareable_url


# for resident in resident_names:
#     file_path = f'/Users/cjyoon/Dropbox/Osler/osler/data/{resident}_schedule.ics'
#     shareable_link = upload_ics_to_drive(file_path)
#     print(f"{resident} schedule: {shareable_link}")